# Data Exploration

This notebook is meant to be a template for explorion of the data collected from trials.

### Please duplicate the notebook to start your work and keep the template intact.

## Before running the notebook
The trial datastore process must be started prior to running the notebook. Make sure you open a terminal, activate the .venv environment, run:
```
python -m launch_local_services
```

Once you are done with working in this notebook, close the terminal window.

## Cogment Setup

In [1]:
import os
import sys

ROOT_DIR = os.path.abspath(os.path.join(os.getcwd(), "../"))
sys.path.append(ROOT_DIR)

from cogment_verse.constants import DEFAULT_CONFIG_DIR, DEFAULT_WORK_DIR, SPEC_FILEPATH, WEB_ACTOR_NAME
from cogment_verse.utils.generate import generate

generate(DEFAULT_WORK_DIR, SPEC_FILEPATH)

/home/william/dev/cogment-verse/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import yaml
from utils import get_trial_data, get_trial_ids

from cogment_verse.specs import EnvironmentSpecs, cog_settings

with open(os.path.join(DEFAULT_CONFIG_DIR, "launch_local_services.yaml"), "r") as f:
    config = yaml.safe_load(f)
port = config["services"]["trial_datastore"]["port"]

TRIAL_DATASTORE_ENDPOINT = f"grpc://localhost:{port}"

## Extract trial ids from datastore

In [4]:
trial_ids = await get_trial_ids(TRIAL_DATASTORE_ENDPOINT)
print(f"Trial ids count: {len(trial_ids)}")
print(f"first 10 trial ids: [{', '.join(trial_ids[:10])}]")

Trial ids count: 1
first 10 trial ids: [admiring_wiles_0_0]


## Extract Trial Data

Example extracting trial info and `samples` for all trials in the datastore.

If only interested in a single or specific set of trial ids, simply pass elements in the list argument `trial_ids` of the function `get_trial_data`.

In [5]:
trial_data = await get_trial_data(TRIAL_DATASTORE_ENDPOINT, trial_ids=[])
test_trial_id = "tender_noether_0_0"

# Trial info about a specific trial ID
info = trial_data[test_trial_id]["trial_info"]

# List of samples for specific Trial ID
samples = trial_data[test_trial_id]["samples"]
print(f"Number of samples in trial: {len(samples)}")

Number of samples in trial: 2175


## Inspect Trial Data

Each trial comes with a set of generic information about the trial. It is an instance of `TrialInfo` class. 

For more information about the `TrialInfo` class, see https://cogment.ai/docs/reference/python#class-trialinfo. 

In [6]:
print(f"Trial ID: {info.trial_id}")
print(f"Trial State: {info.trial_state}")
print(f"Number of steps: {info.sample_count}")

print(f"Environment Implementation: {print(info.parameters.environment_implementation)}")
print(f"Environment Config: {info.parameters.environment_config}")

Trial ID: tender_noether_0_0
Trial State: TrialState.ENDED
Number of steps: 2175
environments.pettingzoo_adapter.AtariEnvironment/pettingzoo.atari.pong_v3
Environment Implementation: None
Environment Config: run_id: "tender_noether_0"
render: true



## Inspect Sample

Each sample from the trial datastore is an instance of the `DatastoreSample` class. 

For more info on the `DatastoreSample` class, see: https://cogment.ai/docs/reference/python#class-datastoresample




In [17]:
sample = samples[0]
print(sample)

environment_specs = EnvironmentSpecs.deserialize(info.parameters.actors[0].config.environment_specs)
action_space = environment_specs.get_action_space()
observation_space = environment_specs.get_observation_space()

# General Sample info
print(f"Trial ID: {sample.trial_id}")
print(f"Trial State: {sample.trial_state}")
print(f"Step: {sample.tick_id}")
print(f"Timestamp: {sample.timestamp}")

# Actor observation, action, reward
print(f"Actors: {list(sample.actors_data.keys())}")
cogment_obs = sample.actors_data["web_actor"].observation
cogment_action = sample.actors_data["web_actor"].action
reward = sample.actors_data["web_actor"].reward

state = observation_space.deserialize(cogment_obs).value
print(f"State: {state}")

action = action_space.deserialize(cogment_action).value
print(f"Action: {action}")

print(f"Reward: {reward}")

DatastoreSample: trial_id = tender_noether_0_0, trial_state = TrialState.RUNNING, tick_id = 0, timestamp = 1682183897894560020 UTC[2023-04-22 17:18:17.894560], nb actors = 2
Trial ID: tender_noether_0_0
Trial State: TrialState.RUNNING
Step: 0
Timestamp: 1682183897894560020
Actors: ['web_actor', 'player_2']
State: [[[  0   0   0  17 255   0]
  [  0   0   0  17 255   0]
  [  0   0   0  17 255   0]
  ...
  [  0   0   0  17 255   0]
  [  0   0   0  17 255   0]
  [  0   0   0  17 255   0]]

 [[  0   0   0  17 255   0]
  [  0   0   0  17 255   0]
  [  0   0   0  17 255   0]
  ...
  [  0   0   0  17 255   0]
  [  0   0   0  17 255   0]
  [  0   0   0  17 255   0]]

 [[  0   0   0  17 255   0]
  [  0   0   0  17 255   0]
  [  0   0   0  17 255   0]
  ...
  [  0   0   0  17 255   0]
  [  0   0   0  17 255   0]
  [  0   0   0  17 255   0]]

 ...

 [[  0   0   0 236 255   0]
  [  0   0   0 236 255   0]
  [  0   0   0 236 255   0]
  ...
  [  0   0   0 236 255   0]
  [  0   0   0 236 255   0]
  [  

In [18]:
human_player = sample.actors_data[WEB_ACTOR_NAME]
opponent = sample.actors_data["player_2"]

print(sample.actors_data)
print(human_player.observation)

{'web_actor': <cogment.datastore.DatastoreActorData object at 0x7f60d5aaf9d0>, 'player_2': <cogment.datastore.DatastoreActorData object at 0x7f60d5aafaf0>}
value {
  dtype: DTYPE_UINT8
  shape: 42336
  raw_data: "\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\022\377\000\000\000\000J\377\000\000\000\000J\377\000\000\000\000J\377\000\000\000\000J\377\000\000\000\000J\377\000\000\000\000J\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\021\377\000\000\000\000\